In [3]:
import pandas as pd
import json
import webbrowser
import time
import requests

decode_serviceKey = 'NwS7t35ZvejHIExKbCymXgmyoLk0WLkZKDslnArD0xyBdUm3ohgWtefXVg25j9L9z3UdyDXXWbfIVjwGfOFArw=='    
ednode_serviceKey = 'NwS7t35ZvejHIExKbCymXgmyoLk0WLkZKDslnArD0xyBdUm3ohgWtefXVg25j9L9z3UdyDXXWbfIVjwGfOFArw%3D%3D'

### getCommercialArea(행정동코드)
def getCommercialArea(signguCd, adongNm):
    ### getDivId(구코드, 찾을 행정동)
    def getDivId():
        #signguCd = '11590' 동작구
        webbrowser.open("http://apis.data.go.kr/B553077/api/open/sdsc2/baroApi")
        time.sleep(1.5) #임시 비동기 처리
            
        url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/baroApi?resId=dong&catId=admi'+'&serviceKey=' + ednode_serviceKey + '&signguCd=' + signguCd + '&type=json'
            
        response_json = requests.get(url, allow_redirects=False)
        response = json.loads(response_json.content)
        responseArr = response.get('body').get('items')
            
        _return = ''
        for list in responseArr:
            if str(list.get('adongNm')) == adongNm :
                _return = list.get('adongCd')
                
            if(_return == ''):
                _return = 'Not found'
            
        return _return
        
    webbrowser.open("http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong")
    time.sleep(2) #임시 비동기 처리
        
    #params
    url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong'
    divId='adongCd'
    key = getDivId()
    params ={'serviceKey' : decode_serviceKey, 'pageNo' : '1', 'numOfRows' : '1000', 'divId' : divId, 'key' : key, 'type' : 'json' }
        
    response_json = requests.get(url, params=params, allow_redirects=False)
    response = json.loads(response_json.content)
    responseArr = response.get('body').get('items')

    _return = pd.DataFrame(responseArr)
        
    #테스트 필요
    page = response.get('body').get('totalCount') // 1000 + 1
    if page > 1 :
        for p in range(1, page):
            params['pageNo'] = str(p)
            add_response_json = requests.get(url, params=params, allow_redirects=False)
            add_response = json.loads(add_response_json.content)
            add_responseArr = add_response.get('body').get('items')
            responseArr = responseArr + add_responseArr
            
    return _return
    
### 필요한 인자가 시군구코드, 찾을행정동
class CommercialArea:
    def __init__(self, signguCd, adongNm):
        self._signguCd = signguCd
        self._adongNm = adongNm
        self._data = getCommercialArea(self._signguCd, self._adongNm)
        
    def getDf(self):
        return self._data
##이런식으로 사용하면 됨
#test = CommercialArea('11590', '상도1동')
#test.getDf()
##리펙토링 마렵네요..

In [4]:
# -*- coding: utf-8 -*-
"""



"""

import pandas as pd

def load_data(univ):
    excel_data = pd.read_csv("대학리스트.csv")
    result_data= excel_data[excel_data['대학']==univ]
    return str(result_data['시군구코드'].values[0]), result_data['행정동이름'].values[0]

    
    
def main():
    a, b = load_data('숭실대학교')
    print(type(str(a)))
    print(type(b))


    
if __name__ =='__main__':
    main()



class LoadUnivData:
    def __init__(self, univ):
        self.univ = univ
        self.gucode, self.dongname = load_data(self.univ)
        
        
    def loadUniv(self):
        return self.gucode, self.dongname
        
        

<class 'str'>
<class 'str'>


In [5]:
from jungeol import LoadUnivData
from jchan import CommercialArea


#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
soongsil = LoadUnivData('숭실대학교') #LoadUnivData 객체 생성
soongsil_gucode, soongsil_dongname = soongsil.loadUniv()

soongsil_CA = CommercialArea(soongsil_gucode, soongsil_dongname) #CommercialArea 객체 생성
soongsil_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(soongsil_CA.getDf().loc[0,:])


bizesId                        11756204
bizesNm                           순에스테틱
brchNm                                 
indsLclsCd                            F
indsLclsNm                        생활서비스
indsMclsCd                          F01
indsMclsNm                      이/미용/건강
indsSclsCd                       F01A03
indsSclsNm                      비만/피부관리
ksicCd                                 
ksicNm                                 
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11590
signguNm                            동작구
adongCd                      1159053000
adongNm                            상도1동
ldongCd                      1159010200
ldongNm                             상도동
lnoCd               1159010200104960022
plotSctCd                             1
plotSctNm                            대지
lnoMnno                             496
lnoSlno                              22
lnoAdr             서울특별시 동작구 상도동 496-22


In [6]:
!pip install folium

In [7]:
import folium
import pandas as pd
import webbrowser

In [8]:
soongsil=soongsil_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
soongsil.to_csv("soongsil.csv",mode="w",encoding='euc-kr')
soongsil_map = pd.read_csv('soongsil.csv', encoding = 'cp949', engine = 'python')
map_CB1 = folium.Map(location = [37.49411614, 126.9543037], zoom_start = 15)
for i, bizesNm in soongsil_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB1)
map_CB1.save('map_CB1.html')
webbrowser.open('map_CB1.html')

True

In [9]:
from jungeol import LoadUnivData
from jchan import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
chungang = LoadUnivData('중앙대학교') #LoadUnivData 객체 생성
chungang_gucode, chungang_dongname = chungang.loadUniv()

chungang_CA = CommercialArea(chungang_gucode, chungang_dongname) #CommercialArea 객체 생성
chungang_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(chungang_CA.getDf().loc[0,:])


bizesId                        10355119
bizesNm                            마이도기
brchNm                                 
indsLclsCd                            D
indsLclsNm                           소매
indsMclsCd                          D05
indsMclsNm                         의복의류
indsSclsCd                       D05A01
indsSclsNm                         일반의류
ksicCd                           G47416
ksicNm                   셔츠 및 기타 의복 소매업
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11590
signguNm                            동작구
adongCd                      1159060500
adongNm                             흑석동
ldongCd                      1159010500
ldongNm                             흑석동
lnoCd               1159010500102210000
plotSctCd                             1
plotSctNm                            대지
lnoMnno                             221
lnoSlno                                
lnoAdr                서울특별시 동작구 흑석동 221


In [10]:
chungang=chungang_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
chungang.to_csv("chungang.csv",mode="w",encoding='euc-kr')
chungang_map = pd.read_csv('chungang.csv', encoding = 'cp949', engine = 'python')
map_CB2 = folium.Map(location = [37.50553583, 126.9546873], zoom_start = 15)
for i, bizesNm in chungang_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB2)
map_CB2.save('map_CB2.html')
webbrowser.open('map_CB2.html')

True

In [11]:
from jungeol import LoadUnivData
from jchan import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
yeonsae= LoadUnivData('연세대학교') #LoadUnivData 객체 생성
yeonsae_gucode, yeonsae_dongname = yeonsae.loadUniv()

yeonsae_CA = CommercialArea(yeonsae_gucode, yeonsae_dongname) #CommercialArea 객체 생성
yeonsae_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(yeonsae_CA.getDf().loc[0,:])


bizesId                        10050747
bizesNm                           제시카키친
brchNm                           이대하늬솔점
indsLclsCd                            Q
indsLclsNm                           음식
indsMclsCd                          Q06
indsMclsNm                           양식
indsSclsCd                       Q06A05
indsSclsNm                      패밀리레스토랑
ksicCd                           I56114
ksicNm                         서양식 음식점업
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11410
signguNm                           서대문구
adongCd                      1141058500
adongNm                             신촌동
ldongCd                      1141011300
ldongNm                             대신동
lnoCd               1141011300100850000
plotSctCd                             1
plotSctNm                            대지
lnoMnno                              85
lnoSlno                                
lnoAdr                서울특별시 서대문구 대신동 85


In [12]:
yeonsae=yeonsae_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
yeonsae.to_csv("yeonsae.csv",mode="w",encoding='euc-kr')
yeonsae_map = pd.read_csv('yeonsae.csv', encoding = 'cp949', engine = 'python')
map_CB3 = folium.Map(location = [37.56376597, 126.9440719], zoom_start = 15)
for i, bizesNm in yeonsae_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB3)
map_CB3.save('map_CB3.html')
webbrowser.open('map_CB3.html')

True

In [13]:
from jungeol import LoadUnivData
from jchan import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
seoul= LoadUnivData('서울대학교') #LoadUnivData 객체 생성
seoul_gucode, seoul_dongname = seoul.loadUniv()

seoul_CA = CommercialArea(seoul_gucode, seoul_dongname) #CommercialArea 객체 생성
seoul_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(seoul_CA.getDf().loc[0,:])


bizesId                        11749509
bizesNm                           가마로강정
brchNm                            녹두거리점
indsLclsCd                            Q
indsLclsNm                           음식
indsMclsCd                          Q05
indsMclsNm                       닭/오리요리
indsSclsCd                       Q05A08
indsSclsNm                    후라이드/양념치킨
ksicCd                           I56193
ksicNm                           치킨 전문점
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11620
signguNm                            관악구
adongCd                      1162073500
adongNm                             대학동
ldongCd                      1162010200
ldongNm                             신림동
lnoCd               1162010200102460049
plotSctCd                             1
plotSctNm                            대지
lnoMnno                             246
lnoSlno                              49
lnoAdr             서울특별시 관악구 신림동 246-49


In [14]:
seoul=seoul_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
seoul.to_csv("seoul.csv",mode="w",encoding='euc-kr')
seoul_map = pd.read_csv('seoul.csv', encoding = 'cp949', engine = 'python')
map_CB4 = folium.Map(location = [37.46860424, 126.9377925], zoom_start = 15)
for i, bizesNm in seoul_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB4)
map_CB4.save('map_CB4.html')
webbrowser.open('map_CB4.html')

True

In [15]:
from jungeol import LoadUnivData
from jchan import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
donguk= LoadUnivData('동국대학교') #LoadUnivData 객체 생성
donguk_gucode, donguk_dongname =donguk.loadUniv()

donguk_CA = CommercialArea(donguk_gucode, donguk_dongname) #CommercialArea 객체 생성
donguk_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(donguk_CA.getDf().loc[0,:])


bizesId                        11334364
bizesNm                             스시효
brchNm                       그랜드앰배서더서울점
indsLclsCd                            Q
indsLclsNm                           음식
indsMclsCd                          Q03
indsMclsNm                       일식/수산물
indsSclsCd                       Q03A03
indsSclsNm                     음식점-초밥전문
ksicCd                           I56113
ksicNm                          일식 음식점업
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11140
signguNm                             중구
adongCd                      1114058000
adongNm                             장충동
ldongCd                      1114014400
ldongNm                           장충동2가
lnoCd               1114014400101860054
plotSctCd                             1
plotSctNm                            대지
lnoMnno                             186
lnoSlno                              54
lnoAdr            서울특별시 중구 장충동2가 186-54


In [16]:
donguk=donguk_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
donguk.to_csv("donguk.csv",mode="w",encoding='euc-kr')
donguk_map = pd.read_csv('donguk.csv', encoding = 'cp949', engine = 'python')
map_CB5 = folium.Map(location = [37.56056745, 127.0021026], zoom_start = 15)
for i, bizesNm in donguk_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB5)
map_CB5.save('map_CB5.html')
webbrowser.open('map_CB5.html')

True

In [34]:
total=pd.concat([soongsil,chungang,yeonsae,seoul,donguk])
total.reset_index(inplace=True)
total.drop('index',axis=1)

,bizesNm,indsLclsNm,indsSclsNm,rdnmAdr,lon,lat
0,순에스테틱,생활서비스,비만/피부관리,서울특별시 동작구 상도로 392-20,126.954304,37.494116
1,구피염색방,생활서비스,여성미용실,서울특별시 동작구 상도로47길 7,126.951374,37.499848
2,엔터투PC방,관광/여가/오락,인터넷PC방,서울특별시 동작구 사당로 30,126.957595,37.494911
3,맛있소면,음식,라면김밥분식,서울특별시 동작구 상도로 295,126.949531,37.501435
4,친구,음식,호프/맥주,서울특별시 동작구 사당로 8,126.955293,37.495314
...,...,...,...,...,...,...
5709,느티나무집,음식,한식/백반/한정식,서울특별시 중구 동호로28길 11,127.003714,37.560266
5710,샤론꽃방.,음식,커피전문점/카페/다방,서울특별시 중구 장충단로 205,127.006619,37.563092
5711,장충김밥,음식,라면김밥분식,서울특별시 중구 장충단로 186,127.006180,37.561233
5712,TGINC,음식,정통양식/경양식,서울특별시 중구 필동로1길 40,126.998563,37.558672


In [46]:
total.to_csv("total.csv",mode="w",encoding='euc-kr')
total_map = pd.read_csv('total.csv', encoding = 'cp949', engine = 'python')
map_CB6 = folium.Map(location = [37.52056745, 127.0021026], zoom_start = 13)
for i, bizesNm in total_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB6)
map_CB6.save('map_CB6.html')
webbrowser.open('map_CB6.html') #합쳐진 데이터 수 렉걸릴 수 있어요 !!

True